# Visualization of Training Results

Этот ноутбук загружает и визуализирует метрики обучения и результаты сравнения моделей GPT-2 и Causal-GPT-2.

In [ ]:
# Импорты
import os
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Путь к папке evaluation
eval_dir = os.path.join(os.path.dirname(os.getcwd()), 'evaluation')
metrics_path = os.path.join(eval_dir, 'metrics.txt')
loss_img_path = os.path.join(eval_dir, 'train_loss_comparison.png')

# Вывод содержимого metrics.txt
print("=== Metrics ===")
with open(metrics_path, 'r') as f:
  print(f.read())

In [ ]:
# Отображение графика кривых обучения
print("=== Training Loss Curves ===")
img = Image.open(loss_img_path)
plt.figure(figsize=(8,6))
plt.imshow(img)
plt.axis('off')
plt.show()

## Вывод

- `metrics.txt` содержит сравнение валидационных потерь и perplexity для обеих моделей.
- График `train_loss_comparison.png` демонстрирует скорость сходимости стандартной GPT-2 и модели с каузальным вниманием.

Дальнейшие шаги:
1. Перейти к оценке качественных метрик (perplexity на большем корпусе, downstream-задачи).
2. Добавить визуализацию эмбеддингов из `causal_embeddings.ipynb` (t-SNE/UMAP).
3. Сравнить генерацию текстов вручную.

In [ ]:
# t-SNE и UMAP визуализация эмбеддингов причин и следствий
import torch
import random
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from implementation.notebooks.causal_embeddings import CausalEmbeddingModel
from sklearn.manifold import TSNE
import umap
import matplotlib.pyplot as plt

# Параметры
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загружаем обученную модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)
causal_model = CausalEmbeddingModel(base_model).to(torch_device)
# загрузка весов (измените путь при необходимости)
# causal_model.load_state_dict(torch.load('../models/causal_emb.pth'))
causal_model.eval()

# Берём случайные 100 пар из BECAUSE 2.0
dataset = load_dataset('causal_bank', split='train[:100]')
pairs = [(ex['cause'], ex['effect']) for ex in dataset]
sample = random.sample(pairs, k=50)

# Функция для вычисления эмбеддинга
@torch.no_grad()
def embed(text):
  enc = tokenizer(text, truncation=True, padding='max_length', max_length=64, return_tensors='pt').to(torch_device)
  emb = causal_model(enc['input_ids'], enc['attention_mask'])
  return emb.cpu().numpy()

# Вычисляем эмбеддинги
emb_c = np.vstack([embed(c) for c,_ in sample])
emb_e = np.vstack([embed(e) for _,e in sample])
X = np.vstack([emb_c, emb_e])
labels = ['cause']*len(emb_c) + ['effect']*len(emb_e)

# t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X)
# UMAP
um = umap.UMAP(n_components=2, random_state=42)
X_umap = um.fit_transform(X)

# Отображение
fig, axes = plt.subplots(1,2, figsize=(12,5))
for ax, coords, title in zip(axes, [X_tsne, X_umap], ['t-SNE', 'UMAP']):
  for lbl, color in [('cause','blue'),('effect','orange')]:
    idx = [i for i,l in enumerate(labels) if l==lbl]
    ax.scatter(coords[idx,0], coords[idx,1], label=lbl, alpha=0.7)
  ax.set_title(title)
  ax.legend()
plt.tight_layout()
plt.show()